In [19]:
from ipywidgets import (FileUpload, HBox, VBox, HTML,
                       Layout, Button, Output, Dropdown,
                       FloatText, Checkbox, Combobox, RadioButtons)

import PIL.Image as Image
import io

from string import Template

from urllib.parse import unquote, urlparse

from IPython.display import display, clear_output, Javascript
import base64

import bokeh
import numpy as np
import pandas as pd

from bokeh.plotting import figure, show
from bokeh.resources import INLINE
from bokeh.models import (CustomJS, ColumnDataSource, 
                          FileInput, Div, Row,
                          TextInput)
from bokeh import events
from bokeh.io import push_notebook, output_notebook

import seeq.addons.plot_digitizer as pdz
from seeq.addons.plot_digitizer._interpolators import interp1d, CubicSpline
from seeq.addons.plot_digitizer._html import (instructions, curve_select_instructions, done_instructions,
                                  region_of_interest_instructions,)

from seeq import spy, sdk
import re

import json

output_notebook(INLINE)
# output_notebook()

Loading BokehJS ...

In [ ]:
def on_click(axis_point_int, func, entry, button):
    func(axis_point_int, entry, button)
    return 

def calibration_table(func):
    x1_entry = FloatText(
        description="X$_{min}$:",
        disabled=True,
        value=None
    )

    x1_button = Button(
        description="Select",
        button_style="warning"
    )
    
    x2_entry = FloatText(
        description="X$_{max}$:",
        disabled=True,
        value=None
    )

    x2_button = Button(
        description="Select",
        button_style="warning"
    )
    
    y1_entry = FloatText(
        description="Y$_{min}$:",
        disabled=True,
        value=None
    )

    y1_button = Button(
        description="Select",
        button_style="warning"
    )
    
    y2_entry = FloatText(
        description="Y$_{max}$:",
        disabled=True,
        value=None
    )

    y2_button = Button(
        description="Select",
        button_style="warning"
    )

    _calibration_table = VBox(
        (
            HBox(
                (x1_entry, x1_button)
            ),
            HBox(
                (x2_entry, x2_button)
            ),
            HBox(
                (y1_entry, y1_button)
            ),
            HBox(
                (y2_entry, y2_button)
            )
        ),
        layout=Layout(border='solid')
    )
    
    
    x1_button.on_click(lambda x: on_click(0, func, x1_entry, x))
    x2_button.on_click(lambda x: on_click(1, func, x2_entry, x))
    y1_button.on_click(lambda x: on_click(2, func, y1_entry, x))
    y2_button.on_click(lambda x: on_click(3, func, y2_entry, x))
    return VBox(
        (HTML("""<h4>Calibration</h4>"""), _calibration_table)
    )

In [2]:
ALERTS_ON=False
IS_SELECTING_AXIS_CALIBRATION_POINT=False
FAILED_TO_LOAD=False
REGION_OF_INTEREST = False
CALIBRATION_HAS_FINISHED = False
interpolation_method = 'cubic'
pushed_curve_sets_and_names = {}
loading_error_message = ''

items_api = sdk.ItemsApi(spy.client)
trees_api = sdk.TreesApi(spy.client)
workbooks_api = sdk.WorkbooksApi(spy.client)
scalars_api = sdk.ScalarsApi(spy.client)
assets_api = sdk.AssetsApi(spy.client)

In [7]:
def alert_message(title, body):
    tmplt = Template(
        """
        require(
            ["base/js/dialog"], 
            function(dialog) {
                dialog.modal({
                    title: '$title',
                    body: '$body',
                    buttons: {
                        'Dismiss': {}}
                });
            }
        );
        """
    )
    
    display(
        Javascript(
            tmplt.substitute(title=title, body=body)
        ))

In [1]:
def get_existing_set_names(curve_set_dict):
    try:
        return list(curve_set_dict.keys())
    except:
        return []

def get_existing_curve_names(curve_set_dict, set_name):
    try:
        return list(curve_set_dict[set_name].keys())
    except:
        return []

In [16]:
def loading_decorator(func):
    global FAILED_TO_LOAD
    def inner(*args, **kwargs):
        if FAILED_TO_LOAD:
            return
        else:
            return func(*args, **kwargs)
    return inner

@loading_decorator
def load_workbook_worksheet(url):
    print('loading workbook and worksheet...')
    global wkb_id, wks_id, workbook, worksheet, FAILED_TO_LOAD, loading_error_message
    workbook_id_match = re.search('workbookId=\w+-\w+-\w+-\w+\w+-\w+', url)
    if workbook_id_match is None:
        loading_error_message = 'No workbook ID identified in URL.'
        raise TypeError('No workbook ID identified in URL.')

    wkb_id_end_pos = workbook_id_match.end()
    wkb_id = url[wkb_id_end_pos-36:wkb_id_end_pos]

    worksheet_id_match = re.search('worksheetId=\w+-\w+-\w+-\w+\w+-\w+', url)
    if worksheet_id_match is None:
        loading_error_message = 'No worksheet ID identified in URL.'
        raise TypeError('No worksheet ID identified in URL.')


    wks_id_end_pos = worksheet_id_match.end()
    wks_id = url[wks_id_end_pos-36:wks_id_end_pos]

    workbook = pdz.get_workbook(wkb_id)
    worksheet = pdz.get_worksheet(workbook, wks_id)

    return

@loading_decorator
def load_signals_ids():
    print('loading signals...')
    global signals, signals_ids, display_items, FAILED_TO_LOAD, loading_error_message
    global xsignal_from_scatterplot, ysignal_from_scatterplot
    display_items = worksheet.display_items
    if len(display_items) < 2:
        loading_error_message = 'At least two signals must be displayed in Workbench. Please load two signals and try again. If you believe you have recieved this message in error, raise an issue on GitHub. '
        FAILED_TO_LOAD = True
        return
    signals = display_items.query("Type == 'Signal'")
    signals_ids = ['{} ({})'.format(name, _id) for name, _id in zip(signals.Name.values, signals.ID.values)]
    
    # get the existing x and y axis from scatterplot, if it exists
    workstep = worksheet.current_workstep()
    stores = workstep.get_workstep_stores()
    
    # x axis
    if not stores['sqScatterPlotStore']['xSignal'] is None:
        xsignal_from_scatterplot = stores['sqScatterPlotStore']['xSignal']['id']
        for sig in signals_ids:
            if xsignal_from_scatterplot in sig:
                xsignal_from_scatterplot = sig
                break
        
    else:
        xsignal_from_scatterplot = None
       
    # y axis
    if len(stores['sqScatterPlotStore']['ySignals']) != 0:
        ysignal_from_scatterplot = stores['sqScatterPlotStore']['ySignals'][0]['id']
        for sig in signals_ids:
            if ysignal_from_scatterplot in sig:
                ysignal_from_scatterplot = sig
                break
    else:
        ysignal_from_scatterplot = None
    

    if len(signals_ids) < 2:
        loading_error_message = 'At least two signals must be displayed in Workbench. Please load two signals and try again. If you believe you have recieved this message in error, raise an issue on GitHub. '
        FAILED_TO_LOAD = True
    return
        
@loading_decorator
def load_assets():
    print('loading assets...')
    global asset_name_to_item_id_dict, asset_names, initial_storage_id, FAILED_TO_LOAD, loading_error_message

    asset_name_to_item_id_dict = pdz.get_available_asset_names_to_item_id_dict(worksheet, trees_api)
    asset_names = asset_name_to_item_id_dict.keys()

    if len(asset_names) == 0:
        FAILED_TO_LOAD = True
        loading_error_message = 'No asset can be found. Ensure you have navigated to signals of interest through their asset tree.'
        return
    
    initial_storage_id = pdz.get_plot_digitizer_storage_id(
        asset_name_to_item_id_dict[list(asset_names)[0]], scalars_api, REGION_OF_INTEREST=REGION_OF_INTEREST
    )
    return

@loading_decorator
def load_initial_curve_sets():
    global initial_curve_set_dict, initial_existing_set_names, curve_set_dict, existing_set_names
    
    initial_curve_set_dict = pdz.get_plot_digitizer_storage_dict(initial_storage_id, scalars_api)
    initial_existing_set_names = get_existing_set_names(initial_curve_set_dict)

    curve_set_dict = initial_curve_set_dict
    existing_set_names = initial_existing_set_names
    clear_output()


In [8]:
### load workbook, worksheet etc

# get the url
url = unquote(jupyter_notebook_url)

load_workbook_worksheet(url)
load_signals_ids()
load_assets()
load_initial_curve_sets()

In [9]:
def _add_plot_point(x, y):
    global s, l, x1
    global CALIBRATION_HAS_FINISHED
    global axis_calibration_counter
    
    if not CALIBRATION_HAS_FINISHED:
        if not IS_SELECTING_AXIS_CALIBRATION_POINT:
            with console_output:
                alert_message('No calibration point selected', 
                  """Select a calibration point before continuing"""
                  )
            return
        
        if axis_calibration_counter == 1:
            x1.data_source.data = dict(x=[x], y=[y])
        if axis_calibration_counter == 2:
            x2.data_source.data = dict(x=[x], y=[y])
        if axis_calibration_counter == 3:
            y1.data_source.data = dict(x=[x], y=[y])
        if axis_calibration_counter == 4:
            y2.data_source.data = dict(x=[x], y=[y])
        push_notebook()
        
        return
    
    if CALIBRATION_HAS_FINISHED and IS_SELECTING_AXIS_CALIBRATION_POINT:
        
        if axis_calibration_counter == 1:
            x1.data_source.data = dict(x=[x], y=[y])
        if axis_calibration_counter == 2:
            x2.data_source.data = dict(x=[x], y=[y])
        if axis_calibration_counter == 3:
            y1.data_source.data = dict(x=[x], y=[y])
        if axis_calibration_counter == 4:
            y2.data_source.data = dict(x=[x], y=[y])
        push_notebook()
        
        return
    
    try:
        ox, oy = list(s.data_source.data['x'].values), list(s.data_source.data['y'].values)
    except AttributeError:
        ox, oy = s.data_source.data['x'], s.data_source.data['y']
        
    # check to make sure strictly increasing or decreasing
    alert_bad_point = False
    if len(ox) >= 2 and not REGION_OF_INTEREST:
        if x - ox[-1] < 0:
            alert_bad_point = True
            
        
    if alert_bad_point and ALERTS_ON:
        with console_output:
            alert_message('Point not allowed.', 
                      """Selected X values must be strictly increasing in order to define a valid function."""
                      )
        return 
    
    ox.append(x)
    oy.append(y)

    if len(ox) > 2 and not REGION_OF_INTEREST:
        # interpolation display
        if interpolation_method == 'cubic':
            order = np.argsort(ox)
            ox = np.array(ox)[order]
            oy = np.array(oy)[order]
            cs = CubicSpline(ox, oy)
            ox3 = np.linspace(min(ox), max(ox), 500)
            oy3 = cs(ox3)
            ox, oy, ox3, oy3 = list(ox), list(oy), list(ox3), list(oy3)
        elif interpolation_method == 'linear':
            order = np.argsort(ox)
            ox = np.array(ox)[order]
            oy = np.array(oy)[order]
            cs = interp1d(ox, oy)
            ox3 = np.linspace(min(ox), max(ox), 500)
            oy3 = cs(ox3)
            ox, oy, ox3, oy3 = list(ox), list(oy), list(ox3), list(oy3)
    
    try:
        s.data_source.data = {'x':ox, 'y':oy}
        l.data_source.data = {'x':ox3, 'y':oy3}
    except NameError:
        s.data_source.data = {'x':ox, 'y':oy}
        l.data_source.data = {'x':ox, 'y':oy}
        
    clear_selection.disabled=False
    
    push_notebook()
    
def _add_point(x, y):
    global REGION_OF_INTEREST
    
    _add_plot_point(x,y)
    
    if not REGION_OF_INTEREST:
        return
    
    
    # adding region of interest below. should draw a dashed line from the most recent point to the original point
    global s, r
    try:
        ox, oy = list(s.data_source.data['x'].values), list(s.data_source.data['y'].values)
    except AttributeError:
        ox, oy = s.data_source.data['x'], s.data_source.data['y']
        
    if (len(ox) == 0):
        return
    else:
        r.data_source.data = {'x':[ox[-1], ox[0]], 'y':[oy[-1], oy[0]]}
        
    push_notebook()
    return
    
def _clear_plot_points():
    global s, l, r
    try:
        s.data_source.data = {'x':[], 'y':[]}
        l.data_source.data = {'x':[], 'y':[]}
        r.data_source.data = {'x':[], 'y':[]}
    except NameError:
        s.data_source.data = {'x':[], 'y':[]}
    clear_selection.disabled=True
    push_notebook()

In [10]:
def digitizer_fig(imgurl):
    # create a Figure object
    p = figure(x_range=(0,1), y_range=(0,1), tools="pan,reset,crosshair,box_zoom")

    
    x = []
    y = []
    s_source = ColumnDataSource(data=dict(x=x, y=y))
    l_source = ColumnDataSource(data=dict(x=x, y=y))
    roi_source = ColumnDataSource(data=dict(x=[], y=[]))

    callback = CustomJS(code="""
        Jupyter.notebook.kernel.execute(`_add_point(${cb_obj.x}, ${cb_obj.y})`)
    """)
    
    # plotting
    p.image_url(url=[imgurl], x=0, y=1, w=1, h=1)
    global s, l, r, x1, x2, y1, y2
    l = p.line(x='x',y='y',source=l_source,color='blue', line_width=4)
    s = p.scatter(x='x',y='y',source=s_source,color='blue', size=8.5)
    calibration_size = 25
    x1 = p.scatter(x=[], y=[], color='blue', size=calibration_size, marker='+', line_width=4)
    x2 = p.scatter(x=[], y=[], color='yellow', size=calibration_size, marker='+', line_width=4)
    y1 = p.scatter(x=[], y=[], color='purple', size=calibration_size, marker='+', line_width=4)
    y2 = p.scatter(x=[], y=[], color='green', size=calibration_size, marker='+', line_width=4)
    r = p.line(x='x', y='y', source=roi_source,color='red', line_width=4, line_dash='dashed')
        
    p.js_on_event(events.Tap, callback)
    p.xaxis.visible = False
    p.yaxis.visible = False
    p.sizing_mode = 'scale_width'
    return p

In [ ]:
@loading_decorator
def load_widgets():
    global xaxis_calibration, yaxis_calibration, interpolation_box
    global file_upload, fig_output, console_output, asset_name, set_name, curve_name
    global x_axis_signal, y_axis_signal, axis_selection_instructions
    global clear_selection, axis_select_box, push_to_seeq, digitize_another_button 
    global push_to_seeq_box, signal_selection_box, user_input, selection_type_radio
    global selection_actions_input_box, cubic_interpolation_checkbox
    global linear_interpolation_checkbox
    
    xaxis_calibration = [(None, None), (None, None)]
    yaxis_calibration = [(None, None), (None, None)]


    # widgets
    file_upload = FileUpload(accept='.png', multiple=False)

    fig_output = Output()
    fig_output.layout = Layout(width='60%')

    def func(axis_point_int, entry, button): 
        global axis_calibration_counter
        global IS_SELECTING_AXIS_CALIBRATION_POINT
        axis_calibration_counter = axis_point_int + 1
        
        if IS_SELECTING_AXIS_CALIBRATION_POINT and button.description != 'Approve' and ALERTS_ON:
            alert_message('Finish axis selection before continuing to the next', 
                      """You are currently in the process of selecting another axis calibration step. Finish before proceeding."""
                     )
            return
            
        
        
        if button.description == 'Select':
            button.description = 'Approve'
            button.button_style = 'success'
            axis_selection_instructions.value = instructions[::-1][axis_calibration_counter]
            entry.disabled=False
            IS_SELECTING_AXIS_CALIBRATION_POINT = True
        elif button.description == 'Approve':
            out = on_axis_point_approval(entry.value)
            if out is not None:
                # case where the image is not yet uploaded (warning occurs in on_axis_point_approval)
                button.description = 'Select'
                button.button_style = 'warning'
                IS_SELECTING_AXIS_CALIBRATION_POINT = True
                return
            button.description = 'Edit'
            button.button_style = 'info'
            entry.disabled=True
            IS_SELECTING_AXIS_CALIBRATION_POINT = False
        else:
            # edit case
            axis_selection_instructions.value = instructions[::-1][axis_calibration_counter]
            button.description = 'Approve'
            button.button_style = 'success'
            entry.disabled=False
            IS_SELECTING_AXIS_CALIBRATION_POINT = True

    console_output = Output()

    asset_name = Dropdown(    
        options=list(asset_names),
        value=list(asset_names)[0],
        description='Asset:',
        disabled=True,
    )

    set_name = Combobox(
        value=None,
        placeholder='',
        options=initial_existing_set_names,
        description='Curve Set:',
        disabled=True
    )

    curve_name = Combobox(
        value=None,
        placeholder='',
        options=[],
        description='Curve Name:',
        disabled=True
    )

    x_axis_signal = Dropdown(
        options=signals_ids,
        value=xsignal_from_scatterplot,
        description='X-Axis:',
        disabled=False,
    )

    y_axis_signal = Dropdown(
        options=signals_ids,
        value=ysignal_from_scatterplot,
        description='Y-Axis:',
        disabled=False,
    )

    axis_selection_instructions = HTML(instructions[-1])

    clear_selection = Button(
        description='Clear Selection',
        disabled=True,
        button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Select',
        layout=Layout(width='50%', positioning='left')
    )
    
    selection_type_radio = RadioButtons(
        options=['Digitize Curve', 'Region of Interest'],
        value='Digitize Curve',
        description='',
        disabled=True,
        layout=Layout(width='50%', positioning='right')
    )
    
    linear_interpolation_checkbox = Checkbox(
        value=False,
        description='Linear Interpolation',
        disabled=True,
        indent=False,
        layout=Layout(width='20%', positioning='left')
    )
    
    cubic_interpolation_checkbox = Checkbox(
        value=True,
        description='Cubic Spline Interpolation',
        disabled=True,
        indent=False,
        layout=Layout(width='20%', positioning='left')
    )

    push_to_seeq = Button(
        description='Push to Seeq',
        disabled=True,
        button_style='success', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Push to seeq',
        icon='check', # (FontAwesome names without the `fa-` prefix),
        layout=Layout(width='50%',)
    )

    digitize_another_button = Button(
        description='New Curve/ROI',
        disabled=True,
        button_style='info', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Digitize another curve or add a new region of interest (using the same axis calibration)',
        layout=Layout(width='50%',)
    )

    push_to_seeq_box = VBox(
        ( 
            HBox(
                (push_to_seeq, digitize_another_button),
                layout=Layout(justify_content='center', width='100%')
            ),
        ),
        layout=Layout(justify_content='flex-end',min_height='100px')
    )
    
    interpolation_box = HBox(
        (linear_interpolation_checkbox, cubic_interpolation_checkbox)
    )

    selection_actions_input_box = HBox(
        (clear_selection, selection_type_radio), 
        layout=Layout(justify_content='space-between', width='100%')
    )

    signal_selection_box = VBox(
        (x_axis_signal, y_axis_signal),
        layout=Layout(justify_content='flex-start', width='100%')
    ) 

    user_input = VBox(
        (
            calibration_table(func),
            signal_selection_box,
            asset_name, 
            set_name, 
            curve_name, 
            selection_actions_input_box,
            push_to_seeq_box,
        ),
        layout=Layout(width='35%')
    )
    return

load_widgets()

In [18]:
# actions

def on_clear_selection(*args):
    _clear_plot_points()
    clear_selection.disabled=True
    

def on_file_upload(change):
    file_upload.disabled = True
    _bytes = file_upload.value[list(file_upload.value.keys())[-1]]['content']
    img = base64.b64encode(_bytes).decode("utf-8") 
    
    global imgurl, fig
    
    imgurl = 'data:image/png;base64,'+img
    fig = digitizer_fig(imgurl)
    with fig_output:
        clear_output()
        show(fig, notebook_handle=True)
    axis_selection_instructions.value = instructions[-2]
    asset_name.disabled=False
    set_name.disabled=False
    curve_name.disabled=False
    with console_output:
        clear_output()
        print('Image loaded.')
        
    linear_interpolation_checkbox.disabled=False
    cubic_interpolation_checkbox.disabled=False
        
axis_calibration_counter = 0 

def on_axis_point_approval(value):
    global axis_calibration_counter, imgurl, s
    global xaxis_calibration, yaxis_calibration
    global CALIBRATION_HAS_FINISHED
    
    try:
        s
    except NameError:
        alert_message('No image loaded', 
                      """You must upload an image before you can calibrate axes."""
                     )
        return "stop"
        
    if axis_calibration_counter == 1:
        # check to make sure unique
        if (value == xaxis_calibration[1][1]) and ALERTS_ON:
            alert_message(
                'Axis calibration points cannot match.',
                'Your input of x-axis value was {}, though you have already selected {}. Please clear selection and try again'.format(value, value)
            )
            return
        
        # print selection
        with console_output:
            clear_output()
            print('Selected FIRST x-axis calibration point at plot x-value = {}'.format(value))
        
        xaxis_calibration[0] = (x1.data_source.data['x'][0], value)
    elif axis_calibration_counter == 2:
        
        # check to make sure unique
        if (value == xaxis_calibration[0][1]) and ALERTS_ON:
            alert_message(
                'Axis calibration points cannot match.',
                'Your input of x-axis value was {}, though you have already selected {}. Please clear selection and try again'.format(value, value)
            )
            return
        # print selection
        with console_output:
            clear_output()
            print('Selected SECOND x-axis calibration point at plot x-value = {}'.format(value))
        
        xaxis_calibration[1] = (x2.data_source.data['x'][0], value)
        
    elif axis_calibration_counter == 3:
        # check to make sure unique
        if (value == yaxis_calibration[1][1]) and ALERTS_ON:
            alert_message(
                'Axis calibration points cannot match.',
                'Your input of y-axis value was {}, though you have already selected {}. Please clear selection and try again'.format(value, value)
            )
            return
        
        # print selection
        with console_output:
            clear_output()
            print('Selected FIRST y-axis calibration point at plot y-value = {}'.format(value))
        
        yaxis_calibration[0] = (y1.data_source.data['y'][0], value)
    elif axis_calibration_counter == 4:
        
        # check to make sure unique
        if (value == yaxis_calibration[0][1]) and ALERTS_ON:
            alert_message(
                'Axis calibration points cannot match.',
                'Your input of y-axis value was {}, though you have already selected {}. Please clear selection and try again'.format(value, value)
            )
            return
        # print selection
        with console_output:
            clear_output()
            print('Selected SECOND y-axis calibration point at plot y-value = {}'.format(value))
        
        yaxis_calibration[1] = (y2.data_source.data['y'][0], value)
        

    _clear_plot_points()
    
    if (np.array((xaxis_calibration, yaxis_calibration)).flatten() == None).any():
        return
    else:
        axis_selection_instructions.value = curve_select_instructions
        CALIBRATION_HAS_FINISHED = True
        push_to_seeq.disabled = False
        selection_type_radio.disabled = False
    
def on_digitize_another(*args):
    global workbook, worksheet, axis_selection_instructions
    axis_selection_instructions.value = curve_select_instructions
    with console_output:
        clear_output()
        print('Reloading workbook...')
    workbook = pdz.get_workbook(wkb_id)
    with console_output:
        clear_output()
        print('Reloading worksheet...')
    worksheet = pdz.get_worksheet(workbook, wks_id)
    with console_output:
        clear_output()
        print('Clearing previous selection...')
    _clear_plot_points()
    push_to_seeq.disabled=False
    selection_type_radio.disabled=False
    digitize_another_button.disabled=True
    with console_output:
        clear_output()
        print('Ready for next curve selection.')
    return
    
        
def on_push_to_seeq(*args):
    if '"' in set_name.value:
        alert_message(
            'Character " is not allowed in set name.',
            """Please use inch, for example, instead."""
        )
        return
    
    if '"' in curve_name.value:
        alert_message(
            'Character " is not allowed in curve name.',
            """Please use inch, for example, instead."""
        )
        return
    
    if "'" in set_name.value:
        alert_message(
            "Character ' is not allowed in set name.",
            """Please use foot, for example, instead."""
        )
        return
    
    if '"' in curve_name.value:
        alert_message(
            "Character ' is not allowed in curve name.",
            """Please use foot, for example, instead."""
        )
        return
    
    set_name.value = set_name.value
    curve_name.value = curve_name.value
    
    if (len(s.data_source.data['x']) < 2 or len(s.data_source.data['y']) < 2) and ALERTS_ON:
        alert_message('No curve/region selected', 
                      """There are {} points selected. When digitizing, you must select at least 3 points. Clear selection and try again.""".format(
                          min((len(s.data_source.data['x']), len(s.data_source.data['y'])))
                      )
                     )
        return
    
    if (x_axis_signal.value is None or x_axis_signal.value == "") and ALERTS_ON:
        alert_message('No x-axis signal (dependent variable) selected.', 'You must select an x-axis (depenedent variable) signal.')
        return
        
    if (y_axis_signal.value is None or y_axis_signal.value == "") and ALERTS_ON:
        alert_message('No y-axis signal (dependent variable) selected.', 'You must select an y-axis (depenedent variable) signal.')
        return
    
    if (x_axis_signal.value == y_axis_signal.value) and ALERTS_ON:
        alert_message('x-axis signal and y-axis signal are the same.', 'You must select two different signals for X and Y.')
        return
    
    clear_selection.disabled = True
    
    global curve_selection_points
    
    with console_output:
        clear_output()
        print('Pushing to Seeq...')
    
    with console_output:
        clear_output()
        print('Calibrating selection by axes...')
    
    curve_selection_points = pd.DataFrame(data=dict(x=s.data_source.data['x'], y=s.data_source.data['y']))
        
    # convert from calibration
    x_cal_array = np.array(xaxis_calibration)
    x_diff = np.diff(x_cal_array, axis=0).flatten()
    x_min = np.min(x_cal_array, axis=0)
    curve_selection_points['real_x'] = ((curve_selection_points['x'] - x_min[0])/(x_diff[0]))*(x_diff[1]) + x_min[1]

    y_cal_array = np.array(yaxis_calibration)
    y_diff = np.diff(y_cal_array, axis=0).flatten()
    y_min = np.min(y_cal_array, axis=0)
    curve_selection_points['real_y'] = ((curve_selection_points['y'] - y_min[0])/(y_diff[0]))*(y_diff[1]) + y_min[1]
    
    if (curve_name.value == '' or set_name.value == '') and ALERTS_ON:
        
        alert_message('No curve set / curve name specified.', 'Please enter a curve set name and curve name.')
        return
        
    if (set_name.value in list(pushed_curve_sets_and_names.keys())):
        with console_output:
            clear_output()
            print('Checking previous digitizations...')
        if (curve_name.value in pushed_curve_sets_and_names[set_name.value]) and ALERTS_ON:
            alert_message('Curve set / curve name has already been pushed to Seeq.', 'Please specify a new curve set/name and try again.')
            return
        
    push_to_seeq.disabled = True
    selection_type_radio.disabled = True
        
    with console_output:
        clear_output()
        print('Retrieving asset...')
    
    asset_id = asset_name_to_item_id_dict[asset_name.value]
    
    with console_output:
        clear_output()
        print('Updating pltdgtz property...')
        print(asset_id)
    
    storage_id = pdz.get_plot_digitizer_storage_id(asset_id, scalars_api, REGION_OF_INTEREST)
    
    pdz.update_pltdgtz_property(
        storage_id, 
        curve_selection_points, 
        set_name.value, 
        curve_name.value, 
        scalars_api
    )

    try:
        pushed_curve_sets_and_names[set_name.value].append(curve_name.value)
    except KeyError:
        pushed_curve_sets_and_names.update({set_name.value:[curve_name.value]})

    with console_output:
        clear_output()
        print('Pushing formula...')
        
        
    # get the x-axis id
    x_match = re.search('(\w+-\w+-\w+-\w+\w+-\w+)', x_axis_signal.value)
    x_axis_id = x_axis_signal.value[x_match.start():x_match.end()]
    # get the y-axis id
    y_match = re.search('(\w+-\w+-\w+-\w+\w+-\w+)', y_axis_signal.value)
    y_axis_id = y_axis_signal.value[y_match.start():y_match.end()]

    formula_push_results = pdz.create_and_push_formula(
        curve_set=set_name.value, 
        curve_name=curve_name.value, 
        storage_id=storage_id, 
        x_axis_id=x_axis_id,
        y_axis_id=y_axis_id,
        REGION_OF_INTEREST=REGION_OF_INTEREST,
        interpolation_method=interpolation_method
    )
    
    curve_set_asset_id = pdz.get_curve_set_asset_id(asset_id, set_name.value, trees_api, assets_api)
    
    # update asset
    pdz.add_item_to_asset(
        asset_id=curve_set_asset_id, item_name=formula_push_results.Name.values[0], 
        item_id=formula_push_results.ID.values[0], trees_api=trees_api,
        overwrite=True
    )

    with console_output:
        clear_output()
        print('Updating workstep...')

    pdz.modify_workstep(
        workbook=workbook, worksheet=worksheet, formula_push_results=formula_push_results, 
        trees_api=trees_api, workbooks_api=workbooks_api,
        x_range=(min(curve_selection_points['real_x']), max(curve_selection_points['real_x'])), 
        y_range=(min(curve_selection_points['real_y']), max(curve_selection_points['real_y'])),  
        x_axis_id=x_axis_id, y_axis_id=y_axis_id, REGION_OF_INTEREST=REGION_OF_INTEREST 
    )

    with console_output:
        axis_selection_instructions.value = done_instructions
        clear_output()
        print('Done. You may digitize another curve / add a new region of interest, or, if done, close this window.')
        

    digitize_another_button.disabled=False
            
    return

def on_asset_selection(change):
    if change['type'] == 'change' and change['name'] == 'value':
        set_name.value = ''
        curve_name.value = ''
        
        global curve_set_dict, existing_set_names
        
        with console_output:
            clear_output()
            print('Searching for existing curve sets...')
         
        storage_id = pdz.get_plot_digitizer_storage_id(
            asset_name_to_item_id_dict[change['new']], scalars_api, REGION_OF_INTEREST=REGION_OF_INTEREST
        )    
        
        curve_set_dict = pdz.get_plot_digitizer_storage_dict(storage_id, scalars_api)
        existing_set_names = get_existing_set_names(curve_set_dict)

            
        with console_output:
            clear_output()
            print('Found {} existing curve sets... Select from dropdown, or type new.'.format(len(curve_set_dict)))
        set_name.options = existing_set_names
        
def on_linear_interpolation_select(change):
    global interpolation_method
    if change['type'] == 'change' and change['name'] == 'value':
        
        if change['new']:
            linear_interpolation_checkbox.value = True
            cubic_interpolation_checkbox.value = False
            interpolation_method = 'linear'
        else:
            linear_interpolation_checkbox.value = False
            cubic_interpolation_checkbox.value = True
            interpolation_method = 'cubic'
    _clear_plot_points()
    clear_selection.disabled=True
    return

def on_cubic_interpolation_select(change):
    global interpolation_method
    if change['type'] == 'change' and change['name'] == 'value':
        
        if change['new']:
            linear_interpolation_checkbox.value = False
            cubic_interpolation_checkbox.value = True
            interpolation_method = 'cubic'
        else:
            linear_interpolation_checkbox.value = True
            cubic_interpolation_checkbox.value = False
            interpolation_method = 'linear'
    _clear_plot_points()
    clear_selection.disabled=True
    return
    
        
def on_radio_selection(change):
    global REGION_OF_INTEREST, curve_set_dict, existing_set_names
    if change['type'] == 'change' and change['name'] == 'value':
        
        if change['new'] == 'Region of Interest':
            REGION_OF_INTEREST = True
            axis_selection_instructions.value = region_of_interest_instructions
            # disable interpolation functions
            linear_interpolation_checkbox.disabled = True
            cubic_interpolation_checkbox.disabled = True
            set_name.description='Region Set:'
            curve_name.description='Region Name:'
        else:
            REGION_OF_INTEREST = False
            axis_selection_instructions.value = curve_select_instructions
            # enable interpolation functions
            linear_interpolation_checkbox.disabled = False
            cubic_interpolation_checkbox.disabled = False
            set_name.description='Curve Set:'
            curve_name.description='Curve Name:'
            
            
        _clear_plot_points()
        
        set_name.value = ''
        curve_name.value = ''
        
        
        storage_id = pdz.get_plot_digitizer_storage_id(
            asset_name_to_item_id_dict[asset_name.value], scalars_api, REGION_OF_INTEREST=REGION_OF_INTEREST
        )    
        
        curve_set_dict = pdz.get_plot_digitizer_storage_dict(storage_id, scalars_api)
        existing_set_names = get_existing_set_names(curve_set_dict)
        
        with console_output:
            clear_output()
            print('Setting REGION OF INTEREST to: {}, searching for existing ROIs... found {} existing sets'.format(REGION_OF_INTEREST, len(existing_set_names)))
            
        set_name.options = existing_set_names
        

def on_curveset_selection(change):
    if change['type'] == 'change' and change['name'] == 'value':
        existing_curve_names = get_existing_curve_names(curve_set_dict, change['new'])
        curve_name.options = existing_curve_names
        with console_output:
            clear_output()
            print('Searching for existing curves... found {}'.format(len(existing_curve_names)))

@loading_decorator
def assign_functions():
    
    # assign functions
    file_upload.observe(on_file_upload, names='value')
    clear_selection.on_click(on_clear_selection)
    push_to_seeq.on_click(on_push_to_seeq)
    digitize_another_button.on_click(on_digitize_another)
    asset_name.observe(on_asset_selection)
    set_name.observe(on_curveset_selection)
    selection_type_radio.observe(on_radio_selection)
    linear_interpolation_checkbox.observe(on_linear_interpolation_select)
    cubic_interpolation_checkbox.observe(on_cubic_interpolation_select)
    return

assign_functions()



NameError: name 'load_widgets' is not defined

Error found at line 1 in cell 18.


Button(description='Click to show stack trace', layout=Layout(height='auto', width='auto'), style=ButtonStyle(…

In [12]:
@loading_decorator
def load_app():
    global header, upload_and_start, active_area, vbox_main

    # App Layout
    header = HTML("""<div align="left"><h1>Plot Digitizer</h1>""")

    upload_and_start = HBox((file_upload,))

    active_area = HBox((fig_output, user_input), layout=Layout(justify_content='space-between'))

    vbox_main = VBox(
        (
            header,
            axis_selection_instructions,
            upload_and_start,
            interpolation_box,
            active_area,
            console_output,
        )
    )
    vbox_main.layout = Layout(width='100%', min_height='500px')
    return

load_app()

In [13]:
ALERTS_ON = True
if not FAILED_TO_LOAD:
    display(vbox_main)
else:
    if ALERTS_ON:
        alert_message('Plot digitizer failed to load.', 'Error Message: {}'.format(loading_error_message))
    print(loading_error_message)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>